In [1]:
# Based off tutorial https://mccormickml.com/2019/07/22/BERT-fine-tuning/#3-tokenization--input-formatting

In [2]:
!git clone https://github.com/huggingface/transformers
!pip install transformers

fatal: destination path 'transformers' already exists and is not an empty directory.
fastai 1.0.59 requires nvidia-ml-py3, which is not installed.
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
# Rerun this

from transformers import BertTokenizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import boto3
import os
import sagemaker
import math
from transformers import *
from sklearn.model_selection import train_test_split
from sagemaker.pytorch import PyTorch
# from transformers import BertTokenizer
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

from time import gmtime, strftime 
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.pytorch import PyTorch

In [4]:
# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [5]:
role = sagemaker.get_execution_role() # we are using the notebook instance role for training in this example

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket() # you can specify a bucket name here

In [6]:
L3_df = pd.read_csv('./data/train/supervised/L3train_nonOneHot.csv')
L3_df.head()

,text,category
0,7 WAYS IMMIGRANTS ENRICH OUR ECONOMY AND SOCIE...,pro-immigration
1,A report by the National Academies of Sciences...,pro-immigration
2,Immigrants are also entrepreneurs who create j...,pro-immigration
3,According to a study of the bipartisan immigra...,pro-immigration
4,Given that the average immigrant-owned busines...,pro-immigration


In [7]:
label_dict = {'pro-immigration':0, 'pro-guns':1, 'anti-immigration':2,
       'oppose-medicare':3, 'pro-medicare':4, 'anti-guns':5}

In [8]:
L3_df['category'] = L3_df.category.apply(lambda x: label_dict[x])

In [9]:
L3_df = shuffle(L3_df,random_state = 100)
L3_df.head()

,text,category
3565,This reflects the decline of hunting as a spor...,5
3589,"(Conversely, a 2014 Quinnipiac survey showed j...",5
388,Even though the evidence of immigration assimi...,0
2376,It Will Make Wait Times Worse Medicare for All...,3
1136,"(These inconvenient facts never get mentioned,...",1


In [10]:
L3_df = pd.DataFrame({
    'id':range(len(L3_df)),
    'label':L3_df['category'],
    'alpha':['a']*L3_df.shape[0],
    'text': L3_df['text']
})

L3_df.head()

,id,label,alpha,text
3565,0,5,a,This reflects the decline of hunting as a spor...
3589,1,5,a,"(Conversely, a 2014 Quinnipiac survey showed j..."
388,2,0,a,Even though the evidence of immigration assimi...
2376,3,3,a,It Will Make Wait Times Worse Medicare for All...
1136,4,1,a,"(These inconvenient facts never get mentioned,..."


In [11]:
cutoff = math.floor(len(L3_df) * 0.9)
print(cutoff)

3690


In [12]:
train_df = L3_df[:cutoff]
dev_df = L3_df[cutoff:]
print('Rows in Training:', train_df.shape[0])
print('Rows in Dev:', dev_df.shape[0])

Rows in Training: 3690
Rows in Dev: 411


In [13]:
data_dir = './data/train/supervised/clean' # The folder we will use for storing data
if not os.path.exists(data_dir): # Make sure that the folder exists
    print("Making folder")
    os.makedirs(data_dir)

In [14]:
train_df.to_csv(os.path.join(data_dir, 'train.tsv'), sep='\t', header=True, index=False, columns=train_df.columns)
dev_df.to_csv(os.path.join(data_dir, 'dev.tsv'), sep='\t', header=True, index=False, columns=dev_df.columns)

In [15]:
# Upload to S3
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix="L3")

In [16]:
input_data

's3://sagemaker-us-west-2-496641494145/L3'

In [28]:
def preprocess_test(candidate):
    df = pd.read_csv('./data/test/%s_test.csv' % candidate)
    df.columns = ['id','text']
    df.to_csv('./data/test/cleaned/%s.tsv' % candidate, sep='\t', header=True, index=False, columns=df.columns)

In [31]:
test_data_dir = './data/test/cleaned'
preprocess_test('Biden')
preprocess_test('Sanders')
preprocess_test('Buttigieg')
preprocess_test('Klobuchar')
preprocess_test('Warren')
preprocess_test('Yang')

# Upload to S3
input_data = sagemaker_session.upload_data(path=test_data_dir, bucket=bucket, key_prefix="L3")

# Practice Post-Processing

In [17]:
pd.read_csv(os.path.join(data_dir, 'train.tsv'), sep = "\t", header = 0)

,id,label,alpha,text
0,0,5,a,This reflects the decline of hunting as a spor...
1,1,5,a,"(Conversely, a 2014 Quinnipiac survey showed j..."
2,2,0,a,Even though the evidence of immigration assimi...
3,3,3,a,It Will Make Wait Times Worse Medicare for All...
4,4,1,a,"(These inconvenient facts never get mentioned,..."
5,5,1,a,If only the Holocaust was the only example of ...
6,6,0,a,Related to the welfare argument is the argumen...
7,7,5,a,Then I looked at the actual empirical research...
8,8,0,a,This does not take into account all the jobs t...
9,9,3,a,Use the health care data that is available to ...


In [18]:
labels = L3_df.label.to_list()

In [19]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenized_input = L3_df.text.apply(lambda x: tokenizer.encode(x,add_special_tokens = True, max_length = 128, pad_to_max_length = True))


In [20]:
l = tokenized_input[0]
# l.extend([0] * (128 - len(l)))
# len(l)
len(l)

128

In [22]:
tokenized_input[0]

[101,
 1021,
 3971,
 7489,
 4372,
 13149,
 2256,
 4610,
 1998,
 2554,
 2755,
 1024,
 7489,
 9002,
 2000,
 3105,
 4325,
 1012,
 102,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [24]:
min([len(p) for p in tokenized_input])

128

In [26]:
# Create attention masks
attn_masks = []

for sent in tokenized_input:
    att_mask = [int(token > 0) for token in sent]
    attn_masks.append(att_mask)

In [27]:
max([len(x) for x in attn_masks])

128

In [28]:
all_input_ids = L3_df.id.to_list()

In [30]:
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)

all_input_ids = torch.tensor(all_input_ids)
attn_masks = torch.tensor(attn_masks)
tokenized_input = torch.tensor(tokenized_input)
labels = torch.tensor(labels)

dataset = TensorDataset(tokenized_input, attn_masks)

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [47]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(tokenized_input, labels, random_state=2018, test_size=0.1)
# Do the same for the masks.
train_masks, validation_masks, _, _ = train_test_split(attn_masks, labels, random_state=2018, test_size=0.1)

In [51]:
Counter(train_labels)

Counter({0: 790, 4: 615, 3: 571, 1: 688, 2: 351, 5: 675})

In [52]:
Counter(validation_labels)

Counter({2: 34, 1: 71, 5: 70, 0: 110, 3: 68, 4: 58})

In [45]:
# Convert to Tensors
attn_masks = torch.tensor(attn_masks, dtype=torch.int32)
tokenized_input = torch.tensor(tokenized_input, dtype=torch.int32)
labels = torch.tensor(labels, dtype=torch.int32)
    
dataset = TensorDataset(tokenized_input, attn_masks, labels)

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  app.launch_new_instance()
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [176]:
sampler = RandomSampler(dataset)
dataloader = DataLoader(dataset, sampler=sampler, batch_size=32)
for step, batch in enumerate(dataloader):
    print(batch[0].size()[0])

32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
5


# Now Doing the Remainder of Postprocessing + Training in a .py file

In [32]:
# data path for the SageMaker PyTorch container. We don't need to create an own container. 
container_data_dir = '/opt/ml/input/data/training'
container_model_dir = '/opt/ml/model'

# input arguments for the training script and initial values for some hyperparameters
parameters = {
    'model_type': 'bert',
    'model_name_or_path' : 'bert-base-uncased',
#     'task_name': task_name,
    'data_dir': container_data_dir,
    'model_dir': container_model_dir,
#     'num_train_epochs': 1,
#     'per_gpu_train_batch_size': 64,
#     'per_gpu_eval_batch_size': 64,
#     'save_steps': 150,
#     'logging_steps': 150,
    'do_train': True,
    'num_labels': 6
#     'do_eval': True,
#     'do_lower_case': True
    # you can add more input arguments here
}

In [33]:
# Amazon SageMaker PyTorch framework

train_instance_type = 'ml.p3.2xlarge'

estimator = PyTorch(entry_point='L3_rebuild.py',
                    source_dir = './train_scripts/', # the local directory stores all relevant scripts for modeling
                    hyperparameters=parameters,
                    role=role,
                    framework_version='1.1.0',
                    train_instance_count=1,
                    train_instance_type=train_instance_type
                   )

In [50]:
# launch model training job
input_data = 's3://sagemaker-us-west-2-496641494145/L3'
estimator.fit({'training': input_data})

2020-03-05 08:39:56 Starting - Starting the training job...
2020-03-05 08:39:58 Starting - Launching requested ML instances...
2020-03-05 08:40:56 Starting - Preparing the instances for training.........
2020-03-05 08:42:16 Downloading - Downloading input data
2020-03-05 08:42:16 Training - Downloading the training image...
2020-03-05 08:42:45 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-03-05 08:42:46,571 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-03-05 08:42:46,595 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-03-05 08:42:46,596 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-03-05 08:42:46,827 sagemaker-containers INFO     Module L3_rebuild does not provide a setup.py. 
Generating setup.py
2020-03-05 08:42:46,827 sagema

In [40]:
# predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

# Make some Predictions

In [46]:
tokenized_input.dtype
labels.dtype

torch.int32

In [48]:
test_example  = dev_df[['id','alpha','text']]

In [50]:
predictor.predict(dev_df[['id','alpha','text']].values)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from model with message "<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>
". See https://us-west-2.console.aws.amazon.com/cloudwatch/home?region=us-west-2#logEventViewer:group=/aws/sagemaker/Endpoints/sagemaker-pytorch-2020-03-04-06-29-51-608 in account 496641494145 for more information.

In [51]:
# Figure out how to run this on testing data

In [52]:
predictor.delete_endpoint()

In [ ]:
accuracies = [0.59, 0.64, 0.65, 0.65]

In [47]:
np.argmax([1,2,34,4])

2